<a href="https://colab.research.google.com/github/SpectraGbes/Spectragbes/blob/main/BCSTCAO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

from collections import Counter

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from torch.utils.data import DataLoader, random_split
from torchinfo import summary
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

torch.backends.cudnn.deterministic = True

In [ ]:
if torch.cuda.is_available():
    device = "cuda"

In [ ]:
os.listdir("sea_creatures")

In [ ]:
class ConvertToRGB:
    def __call__(self, img):
        if img.mode != "RGB":
            img = img.convert("RGB")
        return img


transform = transforms.Compose(
    [
        ConvertToRGB(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)

In [ ]:
data_dir = "sea_creatures"
train_dir = os.path.join(data_dir, "train")
classes = datasets.ImageFolder(root=train_dir, transform=transform).classes
print(classes)

In [ ]:
height = 224
width = 224


class ConvertToRGB:
    def __call__(self, img):
        if img.mode != "RGB":
            img = img.convert("RGB")
        return img


transform = transforms.Compose([
    ConvertToRGB(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

print(transform)

In [ ]:
sample_file = "sea_creatures/train/Dolphin/10004986625_0f786ab86b_b.jpg"

image = Image.open(sample_file)

transformed_image = transform(image)
print(transformed_image.shape)

In [ ]:
dataset = datasets.ImageFolder(root=train_dir, transform=transform)
print("Image size", dataset[0][0].shape)
print("Label", dataset[0][1])

In [ ]:
counts = Counter(x[1] for x in tqdm(dataset))
print("The counts dictionary:", counts)


idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}
print("The class_to_idx dictionary:", dataset.class_to_idx)


class_distribution = {idx_to_class[idx]: count for idx, count in counts.items()}
print(class_distribution)

In [ ]:
batch_size = 32
dataset_loader = DataLoader(dataset, batch_size=batch_size)


first_batch = next(iter(dataset_loader))

print(f"Shape of one batch: {first_batch[0].shape}")
print(f"Shape of labels: {first_batch[1].shape}")

In [ ]:
def get_mean_std(loader):
    """Computes the mean and standard deviation of image data.

    Input: a `DataLoader` producing tensors of shape [batch_size, channels, pixels_x, pixels_y]
    Output: the mean of each channel as a tensor, the standard deviation of each channel as a tensor
            formatted as a tuple (means[channels], std[channels])"""

    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in tqdm(loader):
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_squared_sum += torch.mean(data**2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches

    std = (channels_squared_sum / num_batches - mean**2) ** 0.5

    return mean, std


mean, std = get_mean_std(dataset_loader)

print(f"Mean: {mean}")
print(f"Standard deviation: {std}")

In [ ]:
transform_norm = transforms.Compose(
    [
        ConvertToRGB(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ]
)
print(transform_norm)

In [ ]:
norm_dataset = datasets.ImageFolder(root=train_dir, transform=transform_norm)

print("Image size", norm_dataset[0][0].shape)
print("Label", norm_dataset[0][1])

In [ ]:
g = torch.Generator()
g.manual_seed(42)

train_dataset, val_dataset = random_split(norm_dataset, [0.8, 0.2], generator=g)

print("Training data set size:", len(train_dataset))
print("Validation data set size:", len(val_dataset))

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4)
)

print(model)

In [ ]:
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4)
)
summary(model, input_size=(batch_size, 3, height, width))

In [ ]:
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4)
)


print(model)

In [ ]:
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4)
)

summary(model, input_size=(batch_size, 3, height, width))

In [ ]:
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4),
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4),
    nn.Flatten()
)


print(model)

In [ ]:
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4),
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4),
    nn.Flatten()
)

summary(model, input_size=(batch_size, 3, height, width))

In [ ]:
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4),
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4),
    nn.Flatten(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=64*3*3, out_features=500),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=500, out_features=9)

print(model)

In [ ]:
model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4),
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4),
    nn.Flatten(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=64*3*3, out_features=500),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=500, out_features=9)

summary(model, input_size=(batch_size, 3, height, width))

In [ ]:
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)


model.to(device)

In [ ]:

from training import predict, train
epochs = 10

train(model, optimizer, loss_fn, train_loader, val_loader, epochs=10, device=device)

In [ ]:

probabilities = predict(model, val_loader, device)

predictions = torch.argmax(probabilities, dim=1)

print("Number of predictions:", predictions.shape)

In [ ]:
targets = []

for _, labels in tqdm(val_loader):
    targets.extend(labels.tolist())

In [ ]:

fig, ax = plt.subplots(figsize=(10, 6))

cm = confusion_matrix(targets, predictions.cpu())


classes = datasets.ImageFolder(root=train_dir, transform=transform).classes

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(cmap=plt.cm.Blues, xticks_rotation="vertical", ax=ax)

In [ ]:
test_dir = os.path.join(data_dir, "test")

test_dataset = datasets.ImageFolder(root='sea_creatures/test', transform=transform)

print("Number of test images:", len(test_dataset))

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:

test_probabilities = predict(model, test_loader, device)

test_predictions = torch.argmax(test_probabilities, dim=1)

print("Number of predictions:", test_predictions.shape)

In [ ]:
test_classes = [classes[i] for i in test_predictions]

print("Number of class predictions:", len(test_classes))

In [ ]:
import matplotlib.pyplot as plt
import random


sample_indices = random.sample(range(len(test_loader.dataset.samples)), 12)


fig, axes = plt.subplots(4, 3, figsize=(20, 10))


for ax, idx in zip(axes.flatten(), sample_indices):
    image_path = test_loader.dataset.samples[idx][0]
    img = Image.open(image_path)


    ax.imshow(img)
    ax.axis('off')

    predicted_class = test_classes[idx]

    ax.set_title(f"Predicted: {predicted_class}", fontsize=14)

plt.tight_layout()